[REF](https://www.kaggle.com/c/optiver-realized-volatility-prediction/discussion/279170).

Here I also include 2 modes of training: *single-stock* and *multi-stock*. *multi-stock* training works really fast and you can get decent results after 20 minutes. In this mode train batch includes targets for all stocks in time_id and you have only 3830 training samples. In *single-stock* mode input is still the same and includes data from all stocks, but it also has single stock_id as input and only single stock target for it is predicted. This way batch contains much more diverse (stock_id, time_id) pairs, and I believe this diversity is important to get better score.

Future Scope:
- stock attention placement (before/after RNN and internal implementation
- feature normalization (something with volumes),
- network dimensions, batch size, lr, etc.

**Model Summary:**

1.	Conv1D(21→32, k=3, s=3)
•	Why? Learns local time-window patterns (e.g. short-term momentum or spread shifts) while immediately coarsening 600→200.
2.	GELU → LayerNorm
•	Why? Smooth nonlinearity + per-stock+channel normalization to stabilize feature distributions before/after convolution.
3.	Conv1D(32→32, k=1)
•	Why? A “pointwise” mixing of the 32 channels, analogous to a 1×1 conv in vision models. Refines features at each time step.
4.	StockAttention
•	Why? Learns which stocks’ 200×32 representations should inform one another in this time bucket.
•	A full 112×112 weight matrix allows every stock’s latent representation to borrow information from every other.
5.	Stock Embedding
•	Why? Provides a learned “bias” per stock so the network can distinguish them beyond their raw order-book features.
6.	GRU (32→32)
•	Why? Captures the sequential dynamics within each stock’s 200-step series, integrating patterns over time.
7.	TimeAttention
•	Why? Rather than simply averaging or taking the final hidden state, the model learns to focus on those time steps most predictive of realized volatility.
8.	Linear → Affine → exp
•	Why? Projects the 32-dim attended vector down to a single log-volatility score, rescales (to match data distribution), and exponentiates back to the volatility domain.


[ INPUT ]
 • X shape = (B, S=112, T′=⌊600/coarsen⌋, F=21)
 • raw book & trade features + engineered waps/log-returns/spreads/imbalances  
 • each feature standardized per stock


[ Conv1d #1 ]
 • in_channels = 21  
 • out_channels = D  (e.g. 32)  
 • kernel = k₁ (e.g. 3), stride = s₁ (e.g. 3)  
 → output shape = (B, S, T₁=⌊T′/s₁⌋, D)


[ GELU activation ]


[ LayerNorm over (stocks S, features D) ]
 • normalize across dims 2 & 3  
 → (B, S, T₁, D)


[ Conv1d #2 ]
 • in_channels = D → out_channels = D  
 • kernel = 1, stride = 1  
 → (B, S, T₁, D)


[ GELU activation ]


[ LayerNorm over (stocks S, features D) ]
 → (B, S, T₁, D)


[ StockAttention ]
 • learnable Wₛ ∈ ℝ^(S×S), bₛ ∈ ℝ^S  
 • softmax over stocks → aₛ scores (shape S)  
 • y = ∑_{j=1..S} x_{…j…} · aₛⱼ + current-stock embedding  
 → (B, S, T₁, D)


[ reshape for RNN ]
 (B, S, T₁, D) → ((B·S), T₁, D)


[ GRU ]
 • input_size = D, hidden_size = D  
 • num_layers, dropout…  
 → outputs ((B·S), T₁, D)


[ reshape back ]
 ((B·S), T₁, D) → (B, S, T₁, D)


[ TimeAttention ]
 • learnable wₜ ∈ ℝ^(T₁)  
 • softmax over time → aₜ scores  
 • compress via ∑_{t=1..T₁} x_{…t…}·aₜ  
 → (B, S, D)


[ Linear → 1 ]
 • Dense(D → 1) + bias + learned scaling & shift  
 • exp(·)  
 → “vol” prediction per stock (shape (B, S))


[ LOSS = RMSPE(vol, target) ]

In [1]:
%%capture
!pip install einops
!pip install lightning --quiet

In [2]:
import gc
import re
import os
import einops
import pandas as pd
import numpy as np
import xarray as xr
import matplotlib.pyplot as plt
import ipywidgets as widgets
from tqdm.auto import tqdm
from sklearn.model_selection import KFold
from joblib import Parallel, delayed

import torch
from torch import nn
import lightning.pytorch as pl
from lightning.pytorch.callbacks import Callback, ModelCheckpoint, EarlyStopping
from lightning.pytorch.loggers import CSVLogger
from lightning.pytorch import Trainer
from torch.optim import Adam
from torch.optim.lr_scheduler import ExponentialLR
from torch.utils.data import Dataset, DataLoader
import torch.nn.functional as F

In [3]:
from google.colab import drive
drive.mount('/content/drive')
data_dir = '/content/drive/MyDrive/Colab Notebooks/RBS DL 2025/PRO/data'

Mounted at /content/drive


In [4]:
n_features = 21
n_stocks = 112
n_seconds = 600
# if coarsen > 1, data will be aggregated per this number of seconds. used to reduce mem usage
coarsen = 3

So the four dimensions are:
	1.	Batch (N)
	2.	Stocks (112 in the competition)
	3.	Time slices (600 seconds divided by your coarsen stride → e.g. 200)
	4.	Features (21 raw+engineered columns: prices, sizes, log‐returns, spreads, etc.)

That 4D structure comes out of our preprocessing in prepare_data, where we:
	•	Read per-stock parquet files,
	•	Align everything on the same time_id×seconds_in_bucket grid,
	•	Engineer the 21 features,
	•	Coarsen and stack them into a big NumPy array or xarray with shape (T, S, secs, F),

then wrap that into PyTorch datasets to yield mini-batches of shape (batch, S, secs, F).

In [5]:
# used later for plotting historical train/val results
class MetricHistory(Callback):
    def __init__(self):
        super().__init__()
        self.train_vol = []
        self.valid_vol = []

    def on_train_epoch_end(self, trainer, pl_module):
        m = trainer.callback_metrics
        self.train_vol.append(m['train/vol_loss'].cpu().item())
        self.valid_vol.append(m['valid/vol_loss'].cpu().item())

In [6]:
csv_logger = CSVLogger(
    save_dir='logs',
    name='optiver'
)

In [7]:
def prepare_data(stock_id, stock_ind, set, time_ids, coarsen, norm, out):
    #load book data
    df_book = pd.read_parquet(f'{data_dir}/book_{set}.parquet/stock_id={stock_id}')
    df_min_second = df_book.groupby('time_id').agg(min_second=('seconds_in_bucket', 'min'))
    df_book = df_book.merge(df_min_second, left_on='time_id', right_index=True) \
        .eval('seconds_in_bucket = seconds_in_bucket - min_second') \
        .drop('min_second', axis=1)
    # load trade data
    df_trade = pd.read_parquet(f'{data_dir}/trade_{set}.parquet/stock_id={stock_id}') \
        .merge(df_min_second, left_on='time_id', right_index=True) \
        .eval('seconds_in_bucket = seconds_in_bucket - min_second') \
        .drop('min_second', axis=1)
    # merge book + trade
    df = pd.merge(df_book, df_trade, on=['time_id', 'seconds_in_bucket'], how='outer')
    df['stock_id'] = stock_id
    # set multi index
    df = df.set_index(['stock_id', 'time_id', 'seconds_in_bucket'])
    # pandas -> xarray
    df = df.to_xarray().astype('float32')
    # processing seconds col to make sure it works fine
    df = df.reindex({'time_id': time_ids, 'seconds_in_bucket': np.arange(n_seconds)})
    # forward fill imputation: if no new quote, old quote stays active
    for name in ['bid_price1', 'bid_price2', 'ask_price1', 'ask_price2',
         'bid_size1', 'bid_size2', 'ask_size1', 'ask_size2']:
        df[name] = df[name].ffill('seconds_in_bucket')
    # wap1/2
    df['wap1'] = (df.bid_price1 * df.ask_size1 + df.ask_price1 * df.bid_size1) / (df.bid_size1 + df.ask_size1)
    df['wap2'] = (df.bid_price2 * df.ask_size2 + df.ask_price2 * df.bid_size2) / (df.bid_size2 + df.ask_size2)
    # log(wap1/2)
    df['log_return1'] = np.log(df.wap1).diff('seconds_in_bucket')
    df['log_return2'] = np.log(df.wap2).diff('seconds_in_bucket')
    df['current_vol'] = (df.log_return1 ** 2).sum('seconds_in_bucket') ** 0.5
    df['current_vol_2nd_half'] = (df.log_return1[..., 300:] ** 2).sum('seconds_in_bucket') ** 0.5
    # downsmapling if coursen > 1
    if coarsen > 1:
        mean_features = ['ask_price1', 'ask_price2', 'bid_price1', 'bid_price2',  'ask_size1', 'ask_size2',
               'bid_size1', 'bid_size2', 'price']
        sum_features = ['size', 'order_count']

        df = xr.merge((df[mean_features].coarsen({'seconds_in_bucket': coarsen}, coord_func='min').mean(),
                       df[sum_features].coarsen({'seconds_in_bucket': coarsen}, coord_func='min').sum(),
                       df[['current_vol', 'current_vol_2nd_half']]))
        df['wap1'] = (df.bid_price1 * df.ask_size1 + df.ask_price1 * df.bid_size1) / (df.bid_size1 + df.ask_size1)
        df['wap2'] = (df.bid_price2 * df.ask_size2 + df.ask_price2 * df.bid_size2) / (df.bid_size2 + df.ask_size2)
        df['log_return1'] = np.log(df.wap1).diff('seconds_in_bucket')
        df['log_return2'] = np.log(df.wap2).diff('seconds_in_bucket')
    # ba spread
    df['spread1'] = df.ask_price1 - df.bid_price1
    # order book slope
    df['spread2'] = df.ask_price2 - df.ask_price1
    df['spread3'] = df.bid_price1 - df.bid_price2
    df['total_volume'] = df.ask_size1 + df.ask_size2 + df.bid_size1 + df.bid_size2
    df['volume_imbalance1'] = df.ask_size1 + df.ask_size2 - df.bid_size1 - df.bid_size2
    df['volume_imbalance2'] = (df.ask_size1 + df.ask_size2 - df.bid_size1 - df.bid_size2) / df.total_volume
    for name in ['bid_size1', 'bid_size2', 'ask_size1', 'ask_size2', 'size', 'order_count', 'total_volume']:
        df[name] = np.log1p(df[name])
        # df[name] = df[name].rank('seconds_in_bucket')
    df['volume_imbalance1'] = np.sign(df['volume_imbalance1']) * np.log1p(abs(df['volume_imbalance1']))

    df = df.fillna({'ask_price1': 1, 'ask_price2': 1, 'bid_price1': 1, 'bid_price2': 1,  'ask_size1': 0, 'ask_size2': 0,
               'bid_size1': 0, 'bid_size2': 0, 'price': 1, 'size': 0, 'order_count': 0, 'wap1': 1, 'wap2': 1,
               'log_return1': 0, 'log_return2': 0, 'spread1': 0, 'spread2': 0, 'spread3': 0, 'total_volume': 0,
               'volume_imbalance1': 0, 'volume_imbalance2': 0, 'current_vol': 0, 'current_vol_2nd_half': 0})
    features = ['ask_price1', 'ask_price2', 'bid_price1', 'bid_price2',  'ask_size1', 'ask_size2',
               'bid_size1', 'bid_size2', 'price', 'size', 'order_count', 'wap1', 'wap2',
               'log_return1', 'log_return2', 'spread1', 'spread2', 'spread3', 'total_volume',
               'volume_imbalance1', 'volume_imbalance2']
    extra = ['current_vol', 'current_vol_2nd_half']

    if norm is not None:
        mean = norm['mean'].sel(stock_id=stock_id)
        std = norm['std'].sel(stock_id=stock_id)
    else:
        mean = df.mean(('time_id', 'seconds_in_bucket')).drop(['current_vol', 'current_vol_2nd_half'])
        std = df.std(('time_id', 'seconds_in_bucket')).drop(['current_vol', 'current_vol_2nd_half'])

    df.update((df - mean) / std)
    df = df.astype('float32')

    out[:, stock_ind] = einops.rearrange(df[features].to_array().values, 'f () t sec -> t sec f')
    return df[extra], {'mean': mean, 'std': std}

In [8]:
# pytorch dataset: feeds data into model while training
class OptiverDataset(Dataset):
    def __init__(self, features_data, extra_data, mode, time_ids):
        self.features_data = features_data
        # targets and current_vol
        self.extra_data = extra_data
        self.time_ids = time_ids
        self.mode = mode

    # tells the number of samples to pytorch
    def __len__(self):
        if self.mode == 'single-stock':
            return len(self.time_ids) * n_stocks
        elif self.mode == 'multi-stock':
            return len(self.time_ids)

    # for a given index i, return the corresponding input features + labels for model training.
    def __getitem__(self, i):
        if self.mode == 'single-stock':
            time_id = self.time_ids[i // n_stocks]
            time_ind = self.extra_data.indexes['time_id'].get_loc(time_id)
            stock_ind = i % n_stocks
            stock_id = self.extra_data.indexes['stock_id'][stock_ind]
            return {
                'data': self.features_data[time_ind], # (112, 600, 21)
                'target': self.extra_data['target'].values[time_ind, stock_ind],  # (1,)
                'current_vol': self.extra_data['current_vol'].values[time_ind, stock_ind],  # (1,)
                'current_vol_2nd_half': self.extra_data['current_vol_2nd_half'].values[time_ind, stock_ind],  # (1,)
                'time_id': time_id,
                'stock_id': stock_id,
                'stock_ind': stock_ind
            }
        elif self.mode == 'multi-stock':
            time_id = self.time_ids[i]
            time_ind = self.extra_data.indexes['time_id'].get_loc(time_id)
            return {
                'data': self.features_data[time_ind], # (112, 600, 21)
                'target': self.extra_data['target'].values[time_ind],  # (112,)
                'current_vol': self.extra_data['current_vol'].values[time_ind],  # (112,)
                'current_vol_2nd_half': self.extra_data['current_vol_2nd_half'].values[time_ind],  # (112,)
                'time_id': time_id,
            }

Time Attention Intuition:

Maybe volatility spikes at the beginning of a window. Or maybe at the end of a window. Fixed pooling (mean, max) can't adapt to this. This TimeAttention layer learns to adapt dynamically based on data patterns.

In [9]:
# not all seconds are equally informative — attention helps model prioritize
class TimeAttention(nn.Module):
    # learn which parts of the time window are important
    def __init__(self, steps):
        super().__init__()
        self.steps = steps
        self.weights = nn.Parameter(torch.zeros(steps))

    # the forward pass - collpases time dimension in a weighted manner
    def forward(self, x):
        # x: (b, st, t, f)
        attn = F.softmax(self.weights, 0)
        x = torch.einsum('b s t f, t -> b s f', x, attn)
        return x

Stock Attention Intuition:
Each stock might borrow information from other correlated stocks.

Example:

If stock_10 and stock_25 usually move together, attention will learn to pull information between them. Helps the model generalize better, especially when market events affect multiple stocks.

In [10]:
# You can experiment with other ideas for stock attention: maybe it could be
# something like MultiHeadAttention module with keys and queries that depends on current input,
# maybe it could be a linear combination of all stocks (full connected layer),maybe you can try sparse softmax

class StockAttention(nn.Module):
    def __init__(self, dim):
        super().__init__()
        self.weight = nn.Parameter(torch.zeros((n_stocks, n_stocks)))
        self.bias = nn.Parameter(torch.zeros(n_stocks))
        self.fc_combine = nn.Linear(dim * 2, dim)

    def forward(self, x):
        # x: (batch, stock, time, feature)
        # compute attention scores across assets
        attn = F.softmax(self.weight + self.bias[None, :], dim = -1) # (st, st)
        y = torch.einsum('b i ..., j i -> b j ...', x, attn)
        x = torch.cat((x, y), -1)
        x = self.fc_combine(x)
        return x

In [11]:
class OptiverModel(pl.LightningModule):
    def __init__(
        self,
        mode: str = 'multi-stock',
        dim: int = 32,
        conv1_kernel: int = 3,
        rnn_layers: int = 2,
        rnn_dropout: float = 0.3,
        n_features: int = 21
    ):
        super().__init__()
        self.save_hyperparameters()

        # stock embedding
        self.stock_emb = nn.Embedding(n_stocks, dim)
        self.stock_emb.weight.data.normal_(0, 0.2)

        # two conv layers over time
        self.conv1 = nn.Conv1d(n_features, dim, conv1_kernel, conv1_kernel)
        self.conv2 = nn.Conv1d(dim, dim, 1, 1)
        self.norm1 = nn.LayerNorm([n_stocks, dim])
        self.norm2 = nn.LayerNorm([n_stocks, dim])

        # single GRU
        self.rnn = nn.GRU(dim, dim, rnn_layers,
                        batch_first=True, dropout=rnn_dropout)

        # **ONLY ONE** time-attention head now
        self.timesteps_attn = TimeAttention(600 // conv1_kernel // coarsen)

        # stock-to-stock attention remains
        self.stock_attn = StockAttention(dim)

        # single output head
        self.fc_out1 = nn.Linear(dim, 1)

        # for optional logging
        self.history = pd.DataFrame()

    def forward(self, x, stock_ind=None):
        # x: (batch, stock, time, feature)
        b = x.shape[0]

        # conv1 on each stock separately
        x = einops.rearrange(x, 'b st t f -> (b st) f t')
        x = self.conv1(x)
        x = einops.rearrange(x, '(b st) f t -> b t st f', st=n_stocks)
        x = F.gelu(x)
        x = self.norm1(x)

        # conv2
        x = einops.rearrange(x, 'b t st f -> (b st) f t')
        x = self.conv2(x)
        x = einops.rearrange(x, '(b st) f t -> b t st f', st=n_stocks)
        x = F.gelu(x)
        x = self.norm2(x)

        # back to (b, st, t, f)
        x = einops.rearrange(x, 'b t st f -> b st t f')

        # stock attention + embedding
        x = self.stock_attn(x)
        x = x + self.stock_emb.weight[None, :, None, :]

        # if single-stock mode, pick out that one
        if self.hparams.mode == 'single-stock':
            x = x[torch.arange(b), stock_ind][:, None]

        # GRU expects (batch*stock, time, feature)
        x = einops.rearrange(x, 'b st t f -> (b st) t f')
        x, _ = self.rnn(x)
        x = einops.rearrange(
            x,
            '(b st) t f -> b st t f',
            st=(n_stocks if self.hparams.mode == 'multi-stock' else 1)
        )

        # **single** time attention → output head
        x = self.timesteps_attn(x)       # (b, st, dim)
        x = self.fc_out1(x)              # (b, st, 1)
        x = x * 0.63393 - 5.762331
        x = torch.exp(x)                 # positive volatility

        # return as dict
        if self.hparams.mode == 'single-stock':
            return {'vol': x[:, 0, 0]}
        else:
            return {'vol': x[..., 0]}

    def training_step(self, batch, batch_ind):
        out = self.common_step(batch, 'train')
        return out

    def validation_step(self, batch, batch_ind):
        return self.common_step(batch, 'valid')

    def common_step(self, batch, stage):
        out = self(batch['data'],
                batch.get('stock_ind', None))

        mask = ~torch.isnan(batch['target'])
        target = torch.where(
            mask,
            batch['target'],
            torch.tensor(1.0, device=self.device)
        )

        # primary RMSPE only
        loss = (((out['vol'] - target) / target) ** 2)[mask].mean().sqrt()

        # log both for checkpointing & metrics.csv
        self.log(f'{stage}/loss',     loss,
                on_step=False, on_epoch=True)
        self.log(f'{stage}/vol_loss', loss,
                on_step=False, on_epoch=True)

        return {
            'loss':   loss,
            'target': batch['target'],
            'vol':    out['vol'].detach(),
            'time_id': batch['time_id']
        }

    def common_epoch_end(self, outs, stage):
        target = torch.cat([x['target'] for x in outs])
        vol    = torch.cat([x['vol']    for x in outs])
        mask   = ~torch.isnan(target)

        rmspe = (((vol - target) / target) ** 2)[mask].mean().sqrt()

        # log epoch‐level RMSPE
        self.log(f'{stage}/rmspe',
                rmspe,
                prog_bar=True,
                on_step=False,
                on_epoch=True)

        # optional local history
        self.history.loc[self.trainer.current_epoch,
                        f'{stage}/rmspe'] = rmspe.item()

    def on_fit_start(self):
        self.history_widget = widgets.Output()
        display(self.history_widget)

    def configure_optimizers(self):
        opt = Adam(self.parameters(), lr=0.001)
        sched = {
            'scheduler': ExponentialLR(opt, 0.93),
            'interval': 'epoch'
        }
        return [opt], [sched]

In [12]:
df_train = pd.read_csv(f'{data_dir}/train.csv')
train_data = np.memmap('/content/drive/MyDrive/Colab Notebooks/RBS DL 2025/PRO/train.npy', 'float16', 'w+',
                       shape=(df_train.time_id.nunique(), n_stocks, n_seconds // coarsen, n_features))

res = Parallel(n_jobs=4, verbose=51)(
    delayed(prepare_data)(stock_id, stock_ind, 'train', df_train.time_id.unique(), coarsen, None, train_data)
    for stock_ind, stock_id in enumerate(df_train.stock_id.unique())
)

train_extra = xr.concat([x[0] for x in res], 'stock_id')
train_extra['target'] = df_train.set_index(['time_id', 'stock_id']).to_xarray()['target'].astype('float32')
train_extra = train_extra.transpose('time_id', 'stock_id')
train_norm = {
    'mean': xr.concat([x[1]['mean'] for x in res], 'stock_id'),
    'std': xr.concat([x[1]['std'] for x in res], 'stock_id')
}

# if you data fits in memory, you can load it entirely from disk, otherwise
# training in single-stock mode could be very slow, though it can be OK for multi-stock mode
train_data = np.array(train_data)

[Parallel(n_jobs=4)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=4)]: Done   1 tasks      | elapsed:    9.1s
[Parallel(n_jobs=4)]: Done   2 tasks      | elapsed:    9.1s
[Parallel(n_jobs=4)]: Done   3 tasks      | elapsed:    9.1s
[Parallel(n_jobs=4)]: Done   4 tasks      | elapsed:    9.1s
[Parallel(n_jobs=4)]: Done   5 tasks      | elapsed:   17.0s
[Parallel(n_jobs=4)]: Done   6 tasks      | elapsed:   17.0s
[Parallel(n_jobs=4)]: Done   7 tasks      | elapsed:   17.0s
[Parallel(n_jobs=4)]: Done   8 tasks      | elapsed:   17.0s
[Parallel(n_jobs=4)]: Done   9 tasks      | elapsed:   27.0s
[Parallel(n_jobs=4)]: Done  10 tasks      | elapsed:   27.0s
[Parallel(n_jobs=4)]: Done  11 tasks      | elapsed:   27.0s
[Parallel(n_jobs=4)]: Done  12 tasks      | elapsed:   27.0s
[Parallel(n_jobs=4)]: Done  13 tasks      | elapsed:   35.9s
[Parallel(n_jobs=4)]: Done  14 tasks      | elapsed:   35.9s
[Parallel(n_jobs=4)]: Done  15 tasks      | elapsed:   36.0s
[Parallel(

In [13]:
cv = KFold(5, shuffle=True, random_state=1)
time_ids = train_extra.indexes['time_id'].values
train_time_ids, val_time_ids = next(cv.split(time_ids))

# multi-stock training
train_ds = OptiverDataset(train_data, train_extra, 'multi-stock', time_ids[train_time_ids])
train_dl = DataLoader(train_ds, batch_size=8, shuffle=True, num_workers=1, pin_memory=True, persistent_workers=True)
val_ds = OptiverDataset(train_data, train_extra, 'multi-stock', time_ids[val_time_ids])
val_dl = DataLoader(val_ds, batch_size=32, shuffle=False, num_workers=1, pin_memory=True, persistent_workers=True)

In [14]:
model = OptiverModel(mode='multi-stock',
                     dim=32,
                     conv1_kernel=1,
                     )

In [15]:
# monitor whatever metric you care about; here I pick valid/vol_loss
checkpoint_cb = ModelCheckpoint(
    monitor='valid/vol_loss',   # our RMSPE on vol
    mode='min',
    save_top_k=1,
    filename='epoch{epoch:02d}-vol{valid/vol_loss:.4f}'
)

In [16]:
early_stop = EarlyStopping(
    monitor="valid/vol_loss",   # the metric to watch
    patience=5,                 # how many epochs to wait for improvement
    mode="min",                 # we want to minimize vol_loss
    strict=True,                # error if monitored metric isn’t found
)

In [17]:
trainer = pl.Trainer(
    devices=1,
    accelerator='gpu',
    precision=16,
    max_epochs=50,
    callbacks=[checkpoint_cb, early_stop],
    logger=csv_logger
)

/usr/local/lib/python3.11/dist-packages/lightning/fabric/connector.py:571: `precision=16` is supported for historical reasons but its usage is discouraged. Please set your precision to 16-mixed instead!
INFO: Using 16bit Automatic Mixed Precision (AMP)
INFO:lightning.pytorch.utilities.rank_zero:Using 16bit Automatic Mixed Precision (AMP)
INFO: GPU available: True (cuda), used: True
INFO:lightning.pytorch.utilities.rank_zero:GPU available: True (cuda), used: True
INFO: TPU available: False, using: 0 TPU cores
INFO:lightning.pytorch.utilities.rank_zero:TPU available: False, using: 0 TPU cores
INFO: HPU available: False, using: 0 HPUs
INFO:lightning.pytorch.utilities.rank_zero:HPU available: False, using: 0 HPUs


In [ ]:
trainer.fit(model, train_dl, val_dl)

INFO: You are using a CUDA device ('NVIDIA L4') that has Tensor Cores. To properly utilize them, you should set `torch.set_float32_matmul_precision('medium' | 'high')` which will trade-off precision for performance. For more details, read https://pytorch.org/docs/stable/generated/torch.set_float32_matmul_precision.html#torch.set_float32_matmul_precision
INFO:lightning.pytorch.utilities.rank_zero:You are using a CUDA device ('NVIDIA L4') that has Tensor Cores. To properly utilize them, you should set `torch.set_float32_matmul_precision('medium' | 'high')` which will trade-off precision for performance. For more details, read https://pytorch.org/docs/stable/generated/torch.set_float32_matmul_precision.html#torch.set_float32_matmul_precision
INFO: LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
INFO:lightning.pytorch.accelerators.cuda:LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
INFO: 
  | Name           | Type           | Params | Mode 
----------------------------------------------------------
0

Output()

Sanity Checking: |          | 0/? [00:00<?, ?it/s]

/usr/local/lib/python3.11/dist-packages/lightning/pytorch/trainer/connectors/data_connector.py:425: The 'val_dataloader' does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` to `num_workers=11` in the `DataLoader` to improve performance.
/usr/local/lib/python3.11/dist-packages/lightning/pytorch/trainer/connectors/data_connector.py:425: The 'train_dataloader' does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` to `num_workers=11` in the `DataLoader` to improve performance.


Training: |          | 0/? [00:00<?, ?it/s]

...

Validation: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

In [ ]:
# (Optional) see exactly what keys were logged
print("All logged metrics:", trainer.callback_metrics)

# Now print whichever metrics you care about—e.g. the primary volatility loss:
print(f"Final train vol_loss: {trainer.callback_metrics['train/vol_loss']:.5f}")
print(f"Final valid vol_loss: {trainer.callback_metrics['valid/vol_loss']:.5f}")

# If you also want the overall loss:
print(f"Final train loss:     {trainer.callback_metrics['train/loss']:.5f}")
print(f"Final valid loss:     {trainer.callback_metrics['valid/loss']:.5f}")

In [ ]:
# lightning has already saved the best‐ever checkpoint under `checkpoint_cb.best_model_path`
best_score = checkpoint_cb.best_model_score
best_path  = checkpoint_cb.best_model_path

print(f"✔️  Best valid/vol_loss = {best_score:.5f}")
print(f"   (checkpoint saved to: {best_path})")

In [ ]:
best_path  = checkpoint_cb.best_model_path
best_score = checkpoint_cb.best_model_score  # torch.Tensor(0.2118)

# extract the epoch number from the filename
m = re.search(r"epoch(\d+)-", best_path)
best_epoch = int(m.group(1)) if m else None

print(f"✔️  Best epoch by valid/vol_loss = {best_epoch}")
print(f"    Best valid/vol_loss (RMSPE)    = {best_score:.5f}")
print(f"    Checkpoint path: {best_path}")

In [ ]:
pd.Series(F.softmax(model.timesteps_attn.weights, 0).detach()).plot();

In [ ]:
# — 1) Load the raw metrics.csv that Lightning’s CSVLogger wrote —
#    You should already have defined:
#      csv_logger = CSVLogger(...)
#      trainer = pl.Trainer(..., logger=csv_logger)
#    so that csv_logger.log_dir points to the folder containing metrics.csv.

metrics_path = os.path.join(csv_logger.log_dir, "metrics.csv")
metrics = pd.read_csv(metrics_path)

# — 2) Quick sanity check — which columns do you actually have?
print(">>> Logged columns:", metrics.columns.tolist())
print(metrics.head())

# — 3) One row per epoch — group & take the mean of the two per-epoch entries —
#    (Lightning logs train & val in separate rows, so we average them here)
epoch_metrics = (
    metrics
    .groupby("epoch")[["train/vol_loss", "valid/vol_loss"]]
    .mean()
    .reset_index()
)

# — 4) Plot it —
plt.figure(figsize=(8, 4))
plt.plot(
    epoch_metrics["epoch"],
    epoch_metrics["train/vol_loss"],
    label="train vol_loss (RMSPE)",
)
plt.plot(
    epoch_metrics["epoch"],
    epoch_metrics["valid/vol_loss"],
    "--",
    label="valid vol_loss (RMSPE)",
)
plt.xlabel("Epoch")
plt.ylabel("Vol-loss (RMSPE)")
plt.title("Train vs. Valid RMSPE over Epochs")
plt.legend()
plt.show()

In [ ]:
!rm "/content/drive/MyDrive/Colab Notebooks/RBS DL 2025/PRO/train.npy"